In [6]:
import os
import locale
from enum import Enum

import pandas as pd 
import numpy as np 

from matplotlib import pyplot as plt 
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec


locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')
#locale.setlocale(locale.LC_ALL, '')


'it_IT.UTF-8'

In [16]:
# Daily deaths toll.
daily_data_file_path = os.path.join("..","data", "Istat", "dati-giornalieri-comune")
daily_data_file_name = "comune_giorno.csv"
daily_file = os.path.join(daily_data_file_path, daily_data_file_name)

# Aggregate table.
aggregate_tbl_file_path = os.path.join("..","data", "Istat", "tavola-sintetica-16aprile")
aggregate_tbl_file_name = "Tavola sintetica.xlsx"
aggregate_file = os.path.join(aggregate_tbl_file_path, aggregate_tbl_file_name)


In [19]:
daily_df = pd.read_csv(daily_file, sep=',', lineterminator='\n',low_memory=False, encoding='latin-1')
daily_df.set_index("COD_PROVCOM", inplace = True)
(daily_df.shape, daily_df.dtypes)

((849120, 26),
 REG                  int64
 PROV                 int64
 NOME_REGIONE        object
 NOME_PROVINCIA      object
 NOME_COMUNE         object
 DATA_INIZIO_DIFF    object
 CL_ETA               int64
 GE                   int64
 MASCHI_15            int64
 MASCHI_16            int64
 MASCHI_17            int64
 MASCHI_18            int64
 MASCHI_19            int64
 MASCHI_20            int64
 FEMMINE_15           int64
 FEMMINE_16           int64
 FEMMINE_17           int64
 FEMMINE_18           int64
 FEMMINE_19           int64
 FEMMINE_20           int64
 TOTALE_15            int64
 TOTALE_16            int64
 TOTALE_17            int64
 TOTALE_18            int64
 TOTALE_19            int64
 TOTALE_20\r          int64
 dtype: object)

In [20]:
daily_df.head(3)

,REG,PROV,NOME_REGIONE,NOME_PROVINCIA,NOME_COMUNE,DATA_INIZIO_DIFF,CL_ETA,GE,MASCHI_15,MASCHI_16,...,FEMMINE_17,FEMMINE_18,FEMMINE_19,FEMMINE_20,TOTALE_15,TOTALE_16,TOTALE_17,TOTALE_18,TOTALE_19,TOTALE_20\r
COD_PROVCOM,,,,,,,,,,,,,,,,,,,,,
1001,1,1,Piemonte,Torino,Agliè,1 aprile,17,102,0,0,...,0,1,0,0,0,0,0,1,0,0
1001,1,1,Piemonte,Torino,Agliè,1 aprile,18,104,0,0,...,0,0,0,0,0,1,0,0,0,0
1001,1,1,Piemonte,Torino,Agliè,1 aprile,18,105,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
segrate_istat_code = daily_df.loc[daily_df["NOME_COMUNE"] == "Segrate"].index[0]

In [25]:
segrate_istat_code

15205

In [40]:
daily_df.loc[[segrate_istat_code], ["GE"]]['GE'].value_counts()

105    9
221    8
301    7
306    7
122    7
      ..
412    1
403    1
326    1
427    1
428    1
Name: GE, Length: 120, dtype: int64